<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/ann-for-survey-sampling/blob/main/notebooks/ann_paper_sim_study_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libomp-dev
!pip install faiss-gpu

In [81]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
import faiss
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean
res = faiss.StandardGpuResources() 

In [9]:
def kdtree_impute(tree, sample, data, y, x, eps = 0):
  nns = tree.query(sample[:,x], k = 1, eps = eps)
  res = np.mean(data[nns[1]][:, ys], axis = 0)
  return res

## Simulation 3A

Simulation study taken from: Kim, J. K., & Wang, Z. (2018). Sampling Techniques for Big Data Analysis. International Statistical Review, 1, 1–15. https://doi.org/10.1111/insr.12290

In [4]:
np.random.seed(123)
N = 1000000
x1 = np.random.normal(loc=1.0,scale=1.0,size=N)
x2 = np.random.exponential(scale=1.0, size = N)
epsilon = np.random.normal(size=N)

### target variables
y1 = 1 + x1 + x2 + epsilon
y2 = 0.5*(x1 - 1.5)**2 + x2 + epsilon
## propensity scores
p1 = np.exp(x2) / (1 + np.exp(x2))
p2 = np.exp(-0.5 + 0.5*(x2-2)**2) / (1 + np.exp(-0.5 + 0.5*(x2-2)**2))

data = np.column_stack((x1,x2,y1,y2,p1,p2)).astype('float32')
data[:3]

array([[-0.0856306 ,  0.5546646 ,  0.90054107,  1.2432839 ,  0.63521713,
         0.632858  ],
       [ 1.9973454 ,  1.1975824 ,  4.455222  ,  1.5815529 ,  0.7680944 ,
         0.4556015 ],
       [ 1.2829785 ,  0.84342945,  3.0256119 ,  0.76618266,  0.699187  ,
         0.542107  ]], dtype=float32)

In [ ]:
R = 500
xs = [0,1]
ys = [2,3]
ps = [4,5]

results_faiss_500 = np.zeros(shape = (R, 2))
results_faiss_1000 = np.zeros(shape = (R, 2))
results_kdtree_500 = np.zeros(shape = (R, 2))
results_kdtree_1000 = np.zeros(shape = (R, 2))

for r in range(R):
  print(r)
  np.random.seed(r)
  ## big data sample
  big_p1 = np.random.binomial(n=1, p = p1, size = N)
  big_p2 = np.random.binomial(n=1, p = p2, size = N)    
  ## random samples
  s500 = np.random.choice(a = data.shape[0], size = 500, replace = False)
  s1000 = np.random.choice(a = data.shape[0], size = 1000, replace = False)
  ## kdtree (exact)
  kdtree = KDTree(data[big_p1==1][:, xs], leafsize = 10)
  results_kdtree_500[r, :] = kdtree_impute(kdtree, data[s500], data[big_p1==1], ys, xs)
  results_kdtree_1000[r, :] = kdtree_impute(kdtree, data[s1000], data[big_p1==1], ys, xs)
  ## faiss
  big_data = data[big_p1==1][:, xs].copy()
  sam_data_500 = data[s500][:, xs].copy()
  sam_data_1000 = data[s1000][:, xs].copy()
  index_flat = faiss.IndexFlatL2(len(xs))
  gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
  gpu_index_flat.add(big_data)
  D_500, I_500 = gpu_index_flat.search(sam_data_500, k = 1) 
  D_1000, I_1000 = gpu_index_flat.search(sam_data_1000, k = 1) 
  ind_500 = [i[0] for i in I_500]
  ind_1000 = [i[0] for i in I_1000]
  results_faiss_500[r,:]=np.mean(data[big_p1==1][ind_500][:, ys], axis=0)
  results_faiss_1000[r,:]=np.mean(data[big_p1==1][ind_1000][:, ys], axis=0)

In [59]:
expected = np.stack(
    [np.mean(results_kdtree_500, axis=0),
     np.mean(results_faiss_500, axis=0),
     np.mean(results_kdtree_1000, axis=0),
     np.mean(results_faiss_1000, axis=0)
     ]
) 

stderrs =  np.stack(
    [np.std(results_kdtree_500, axis=0),
     np.std(results_faiss_500, axis=0),
     np.std(results_kdtree_1000, axis=0),
     np.std(results_faiss_1000, axis=0)
     ]
)

bias = expected - np.mean(data[:,ys], axis=0)
mse = bias**2 + stderrs**2

print("===== bias =====")
print(bias)
print("===== se =====")
print(stderrs)
print("===== mse =====")
print(mse)

===== bias =====
[[ 0.00424795 -0.00285448]
 [ 0.00378274 -0.00331977]
 [ 0.00329126  0.00082333]
 [ 0.00325391  0.00078617]]
===== se =====
[[0.07272113 0.07345612]
 [0.07295542 0.07315786]
 [0.05202406 0.04801981]
 [0.05191877 0.04749557]]
===== mse =====
[[0.00530641 0.00540395]
 [0.0053368  0.00536309]
 [0.00271734 0.00230658]
 [0.00270615 0.00225645]]


In [63]:
## save results
results = {
    "results_kdtree_500" : pd.DataFrame(results_kdtree_500),
    "results_faiss_500" : pd.DataFrame(results_faiss_500),
    "results_kdtree_1000": pd.DataFrame(results_kdtree_1000),
    "results_faiss_1000": pd.DataFrame(results_faiss_1000),
           }

f = open("kdtree_faiss_500_1000k.pkl","wb")
pickle.dump(results,f)
f.close()

## Simulation 3B

Modified case from: Yang, S., & Kim, J. K. (2019). Nearest neighbour imputation for general parameter estimation in survey sampling. In The Econometrics of Complex Survey Data: Theory and Applications (Vol. 39, pp. 209–234)

In [17]:
def kdtree_impute2(tree, data_obs, data_not, y, x, N, eps = 0):
  nns = tree.query(data_not[:,xs], k = 1, eps = eps)
  res = (np.sum(data_obs[nns[1]][:, ys], axis = 0) + np.sum(data_obs[:, ys], axis = 0)) / N
  return res

In [50]:
np.random.seed(123)
N = 50000
x1 = np.random.uniform(size = N)
x2 = np.random.uniform(size = N)
x3 = np.random.uniform(size = N)
x4 = np.random.normal(size = N)
x5 = np.random.normal(size = N)
x6 = np.random.normal(size = N)
epsilon = np.random.normal(size=N)

### target variables
y1 = -1 + x1 + x2 + epsilon
y2 = -1.5 + x1 + x2 + x3 + x4 + epsilon
y3 = -1.5 + x1 + x2 + x3 + x4 + x5 + x6 + epsilon
y4 = -1 + x1 + x2 + x1**2 + x2**2 -2/3 + epsilon
y5 = -1 + x1 + x2 + x3 + x4 + x1**2 + x2**2 -2/3 + epsilon
y6 = -1 + x1 + x2 + x3 + x4 + x5 + x6 + x1**2 + x2**2 -2/3 + epsilon

## propensity scores
p1 = np.exp(1 + x1 + x2 + x3 + x4 + x5 + x6) / (1 + np.exp(1 + x1 + x2 + x3 + x4 + x5 + x6))
p2 = np.exp(1 + x1 + x2) / (1 + np.exp(1 + x1 + x2))

data = np.column_stack((x1,x2,x3,x4,x5,x6,y1,y2,y3,y4,y5,y6)).astype('float32')
data[:3]

array([[ 0.6964692 ,  0.36086547,  0.20932843,  0.47665665,  0.80429375,
        -0.9894137 ,  0.42543107,  0.61141616,  0.42629617,  0.3740576 ,
         1.0600427 ,  0.87492275],
       [ 0.28613934,  0.22535679,  0.2937351 ,  0.6701647 ,  1.2931948 ,
         1.033963  ,  0.25331086,  0.71721065,  3.0443685 , -0.2806944 ,
         0.6832054 ,  3.010363  ],
       [ 0.22685145,  0.50813043,  0.05571789,  0.5033514 ,  1.5591047 ,
         0.13504769,  0.36729714,  0.42636642,  2.1205187 ,  0.0102886 ,
         0.5693579 ,  2.2635102 ]], dtype=float32)

0.51176731996

In [98]:
R = 500
xs = [0,1,2,3,4,5]
ys = [6,7,8,9,10,11]

results_faiss = np.zeros(shape = (R, 6))
results_kdtree = np.zeros(shape = (R, 6))
results_pmm = np.zeros(shape = (R, 6))

for r in range(R):
  print(r)
  np.random.seed(r)
  ## big data sample
  big_p1 = np.random.binomial(n=1, p = p1, size = N)
  ## data
  data_obs = data[big_p1 == 1].copy()
  data_not = data[big_p1 != 1].copy()
  ## kdtree (exact)
  #kdtree = KDTree(data_obs[:, xs], leafsize = 10)
  #results_kdtree[r, :] = kdtree_impute2(kdtree, data_obs, data_not, ys, xs, N)
  ## predictive mean matching for y1
  X_obs = data_obs[:, xs]
  X_obs = sm.add_constant(X_obs)
  y_obs = data_obs[:, ys[5]]
  ols_y = sm.OLS(y_obs, X_obs).fit()
  X_not = data_not[:, xs]
  X_not = sm.add_constant(X_not)
  y_not = np.dot(X_not, ols_y.params)
  results_pmm[r, 5] = (np.sum(np.array([y_obs[np.argmin(np.sqrt((y_obs - row)**2))] for row in y_not])) + sum(y_obs)) / N
  ## faiss
  data_obs_faiss = data_obs[:, xs].copy()
  data_not_faiss = data_not[:, xs].copy()
  index_flat = faiss.IndexFlatL2(len(xs))
  gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
  gpu_index_flat.add(data_obs_faiss)
  dists, inds = gpu_index_flat.search(data_not_faiss, k = 1) 
  inds_imp = [i[0] for i in inds]
  results_faiss[r,:]= (np.sum(data_obs[inds_imp][:, ys], axis=0) + np.sum(data_obs[:, ys], axis=0)) / N

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [109]:
expected = np.stack(
    [#np.mean(results_kdtree, axis=0),
     np.mean(results_faiss, axis=0)
     ]
) 

stderrs =  np.stack(
    [#np.std(results_kdtree, axis=0),
     np.std(results_faiss, axis=0)
     ]
)

bias = expected - np.mean(data[:,ys], axis=0)
mse = bias**2 + stderrs**2

print("===== bias =====")
print(bias)
print("===== se =====")
print(stderrs)
print("===== mse =====")
print(mse)
print(expected)
print(np.mean(results_pmm[:, 5]))
print(np.std(results_pmm[:, 5]))

===== bias =====
[[849.00727166 140.62519024 178.91123984 762.39777878   2.4278875
    3.98481115]]
===== se =====
[[0.00308666 0.00313541 0.00326809 0.00316385 0.00319099 0.00330361]]
===== mse =====
[[2.74536077e-05 1.14547982e-04 3.43846135e-04 5.00142213e-05
  1.62066949e-04 4.24783205e-04]]
[[0.00473262 0.01751003 0.028455   0.0071545  0.5199319  0.53087687]]
0.511664524975071
0.002141024609639731


## Simulation 3C

Study from Yang, S., & Kim, J. K. (2020). Doubly robust inference when combining probability and non-probability samples with high dimensional. Journal of the Royal Statistical Society. Series B: Statistical Methodology, 82(2), 445–465. https://doi.org/10.1111/rssb.12354

R code (source: https://github.com/shuyang1987/IntegrativeFPM)

```r
set.seed(1234)
## population size
N <- 10000
## x is a p-dimensional covariate
p <- 50
x <- matrix( rnorm(N*p,0,1),N,p)
## y is a continuous outcome 
beta0 <- c(1,1,1,1,1,rep(0,p-4))
y <- cbind(1,x)%*%beta0 + rnorm(N,0,1)
true <- mean(y)
## y2 is a binary outcome
ly2 <- (cbind(1,x)%*%beta0)
ply <- exp(ly2)/(1+exp(ly2))
y2 <- rbinom(N,1,ply)
true2 <- mean(y2)
## A.set is a prob sample: SRS
## sampling probability into A is known when estimation
nAexp <- 1000
probA <- rep(nAexp/N,N)
A.index <- rbinom(N,size = 1,prob = probA)
A.loc <- which(A.index == 1)
nA <- sum(A.index == 1)
sw.A <- 1/probA[A.loc]
x.A <- x[A.loc,]
y.A <- rep(NA,nA) # y is not observed in Sample A
y2.A <- rep(NA,nA)
## B.set is a nonprob sample
## sampling probability into B is unknown when estimation
nBexp <- 2000
alpha0 <- c(-2,1,1,1,1,rep(0,p-4))
probB <- (1+exp(-cbind(1,x)%*%alpha0))^(-1) 
B.index <- rbinom(N,size = 1,prob = probB)
B.loc <- which(B.index == 1)
nB <- sum(B.index)
x.B <- x[B.loc,]
y.B <- y[B.loc]
y2.B <- y2[B.loc]
## combined dataset
y.AB <- c(y.A,y.B)
y2.AB <- c(y2.A,y2.B)
x.AB <- rbind(x.A,x.B)
deltaB <- c(rep(0,nA),rep(1,nB))
sw <- c(sw.A,rep(1,nB))
```